# PENALIZACIONES L1 Y L2

In [18]:
import pandas as pd

df = pd.read_csv("ABBREV.csv")

X = df[['Protein_(g)', 'Lipid_Tot_(g)', 'Carbohydrt_(g)']].values.tolist()
y = df['Energ_Kcal'].tolist()

def normalize(data):
    mean = sum(data) / len(data)
    std = (sum([(x - mean) ** 2 for x in data]) / len(data)) ** 0.5
    return [(x - mean) / std for x in data]

# Normalizar cada columna de X y la variable objetivo y
X_normalized = [[normalize(col) for col in zip(*X)]]
X_normalized = list(zip(*X_normalized[0]))  # Trasponer la lista
y_normalized = normalize(y)


### Penalización L2 (Ridge)

In [19]:
def ridge_penalty(w, X, y, alpha):
    m = len(y)
    predictions = [sum([w_j * x_j for w_j, x_j in zip(w, x)]) for x in X]
    error = [(p - y_i) ** 2 for p, y_i in zip(predictions, y)]
    cost = (1 / m) * sum(error) + alpha * sum([w_j ** 2 for w_j in w])  # Penalización L2
    return cost


In [20]:
def gradient_descent_ridge(X, y, alpha, learning_rate, epochs):
    m = len(y)
    n = len(X[0])
    w = [0] * n  # Inicializar pesos a 0
    for epoch in range(epochs):
        gradients = [0] * n
        for i in range(m):
            prediction = sum([w_j * X[i][j] for j, w_j in enumerate(w)])
            error = prediction - y[i]
            for j in range(n):
                gradients[j] += (2/m) * error * X[i][j]
        w = [w_j - learning_rate * (grad + 2 * alpha * w_j) for w_j, grad in zip(w, gradients)]
    return w


### Penalización L1 (Lasso)

In [21]:
def lasso_penalty(w, X, y, alpha):
    m = len(y)
    predictions = [sum([w_j * x_j for w_j, x_j in zip(w, x)]) for x in X]
    error = [(p - y_i) ** 2 for p, y_i in zip(predictions, y)]
    cost = (1 / m) * sum(error) + alpha * sum([abs(w_j) for w_j in w])  # Penalización L1
    return cost

In [22]:

def gradient_descent_lasso(X, y, alpha, learning_rate, epochs):
    m = len(y)
    n = len(X[0])
    w = [0] * n  # Inicializar pesos a 0
    for epoch in range(epochs):
        gradients = [0] * n
        for i in range(m):
            prediction = sum([w_j * X[i][j] for j, w_j in enumerate(w)])
            error = prediction - y[i]
            for j in range(n):
                gradients[j] += (2/m) * error * X[i][j]
        w = [w_j - learning_rate * (grad + alpha * (1 if w_j > 0 else -1)) for w_j, grad in zip(w, gradients)]
    return w

In [ ]:
# Parámetros para el descenso de gradiente
alpha = 0.1  # Factor de penalización
learning_rate = 0.01
epochs = 1000


w_ridge = gradient_descent_ridge(X_normalized, y_normalized, alpha, learning_rate, epochs)


w_lasso = gradient_descent_lasso(X_normalized, y_normalized, alpha, learning_rate, epochs)

weights_df = pd.DataFrame({
    'Característica': ['Protein_(g)', 'Lipid_Tot_(g)', 'Carbohydrt_(g)'],
    'Peso_L2_Ridge': w_ridge,
    'Peso_L1_Lasso': w_lasso
})

# Mostrar los resultados
weights_df